### **Moondream2**

In [ ]:
from google.colab import drive; drive.mount('/content/drive')

In [7]:
!pip install -U bitsandbytes
!pip install transformers einops
!pip install flash-attn --no-build-isolation

In [33]:
import torch
import random
import requests
import pandas as pd
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [21]:
trnhdf = pd.read_csv("/content/train.csv")
trnhdf = trnhdf[trnhdf["entity_name"] == "height"]
trnhdf.head()

,image_link,group_id,entity_name,entity_value
536,https://m.media-amazon.com/images/I/610bLFQIS3...,442321,height,95.0 centimetre
1740,https://m.media-amazon.com/images/I/512Bph1Ouu...,507467,height,46.5 millimetre
2306,https://m.media-amazon.com/images/I/61+g4VX7ww...,237000,height,71.0 centimetre
2363,https://m.media-amazon.com/images/I/515cEBwzfo...,245959,height,28.0 centimetre
2667,https://m.media-amazon.com/images/I/51K5y5KflQ...,752266,height,308.0 millimetre


In [ ]:
model_id = "vikhyatk/moondream2"
revision = "2024-08-26"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    revision=revision,
    trust_remote_code=True,
    torch_dtype=torch.float16,
).eval().to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)

In [44]:
%%timeit -n 20 -r 4
random_index = random.randint(0, len(trnhdf) - 1)
row = trnhdf.iloc[random_index]
url = row["image_link"]
real_value = row["entity_value"]
response = requests.get(url)
image = Image.open(BytesIO(response.content))
enc_image = model.encode_image(image).to('cuda')
predicted_value = model.answer_question(enc_image, "height?", tokenizer)
print(f"Predicted: {predicted_value}, Real: {real_value}")

Predicted: 65 cm, Real: 155.0 centimetre
Predicted: 21mm, Real: 25.0 millimetre
Predicted: The height of the water bottle is 4.7 cm., Real: 23.0 centimetre
Predicted: 0.34", Real: 123.9 millimetre
Predicted: 10-30 inches, Real: 10.0 inch
Predicted: 1.8cm, Real: 1.8 centimetre
Predicted: 3.50 inches, Real: 4.25 inch
Predicted: 53cm, Real: 53.0 centimetre
Predicted: 40 cm, Real: 40.0 centimetre
Predicted: 120mm, Real: 125.0 millimetre
Predicted: 16.3 cm, Real: 17.0 centimetre
Predicted: 14.96 inches, Real: 14.96 inch
Predicted: 11.64 inches, Real: 13.08 inch
Predicted: 12inch/30cm, Real: 45.0 centimetre
Predicted: 27cm/10.63", Real: 20.0 centimetre
Predicted: 30.5 cm, Real: 30.5 centimetre
Predicted: 2 inches, Real: 2.0 inch
Predicted: 24 cm, Real: 10.0 centimetre
Predicted: 38 cm, Real: 43.0 centimetre
Predicted: 60 cm, Real: 60.0 centimetre
Predicted: 8.26 inches, Real: 8.26 inch
Predicted: 33 inches, Real: 33.0 inch
Predicted: 10 cm, Real: 10.0 centimetre
Predicted: 50 cm, Real: 50.0 